## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Airai,TL,-8.9266,125.4092,72.88,55,28,4.88,scattered clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,44.60,87,1,5.75,clear sky
2,2,Margate,GB,51.3813,1.3862,42.01,78,3,4.00,clear sky
3,3,Markivka,UA,49.5229,39.5705,50.59,67,75,10.09,light rain
4,4,Carnarvon,AU,-24.8667,113.6333,84.76,39,2,8.10,clear sky


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum tempurature you would like for your trip?'))

What is the minimum temperature you would like for your trip?75
What is the maximum tempurature you would like for your trip?90


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                      (city_data_df['Max Temp'] >= min_temp)]

In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID        166
City           166
Country        166
Lat            166
Lng            166
Max Temp       166
Humidity       166
Cloudiness     166
Wind Speed     166
Description    166
dtype: int64

In [55]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [57]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,84.76,clear sky,-24.8667,113.6333,
8,Sorong,ID,86.68,overcast clouds,-0.8833,131.2500,
9,Soure,BR,75.79,overcast clouds,-0.7167,-48.5233,
14,Busselton,AU,75.99,scattered clouds,-33.6500,115.3333,
15,Ternate,ID,83.68,broken clouds,0.8000,127.4000,
25,Buala,SB,83.75,light rain,-8.1450,159.5921,
28,Mahebourg,MU,84.20,broken clouds,-20.4081,57.7000,
29,Mayumba,GA,78.24,light rain,-3.4320,10.6554,
30,Butaritari,KI,81.86,light rain,3.0707,172.7902,
36,Rikitea,PF,77.77,overcast clouds,-23.1203,-134.9692,


In [65]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [71]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))